* Overview of Troubleshooting Application Issues
* Run Application locally in debug mode
* Generating Additional logs in Flask Application
* Review Supervisor and Gunicorn Logs on Servers
* Extract SQL Queries using Flask SQLAlchemy
* Log Message for User Not Found
* Exercise and Solution - Log Message for Course Not Found

* Overview of Troubleshooting Application Issues

1. Review logs (supervisor, gunicorn and Flask logs)
2. Ensure all important events are logged at appropriate level
3. Run application locally in debug mode
4. Reproduce the issues in local environment
5. Log SQL Queries to understand underlying queries

* Run Application locally in debug mode

1. Use `python -f flask run --debug`. We don't need to restart the application when there are changes.
2. This approach is not recommended in non development environments.
3. All the calls to the end points will be logged.

* Generating Additional logs in Flask Application

1. To generate more logs we can use `app.logger`. Also, we need to add logging with relevant levels as per the logging requirements.
2. All the exceptions should be logged using `ERROR` log level.

* Review Supervisor and Gunicorn Logs on Servers

1. Connect to the remote VM using SSH.
2. Go to `/var/log/supervisor` and check for log file.
3. Review the log file to understand the supervisor logs generated.
4. Go to `/var/log/sales-app` and check for log files.
5. Review the log files to understand the application logs generated. These log files are generated by gunicorn. Also, the Flask logs will be forwarded by gunicorn to these log files.

* Extract SQL Queries using Flask SQLAlchemy

1. Update `app.py` before `db.init_app(app)` with below code. All the sql alchemy configurations should be set before this line.

```python
app.config['SQLALCHEMY_ECHO'] = eval(os.environ.get('LOG_SQL_QUERIES', 'False'))
```

2. Launch flask shell as we can validate using flask shell.
3. Run below code snippets to see SQL Queries based on the API calls on models.

```python
User.query.get(1)
User.query.filter(User.email.like('sti%')).all()
```

4. You will start seeing the queries as well as parameter values used.

* Log Message for User Not Found

We need to add additional log messages for the scenarios such as User Not Found. Typically these are logged at `ERROR` level. We will also change the application log level to `INFO`.

1. Change the application log level in `app.py` to `INFO`.
2. Update `users.py` with logging for GET APIs (includes delete as well).

```python
@app.route('/user', methods=['GET', 'POST'])
def user():
    id = request.args.get('id')
    if request.method == 'GET':
        if id:
            user = User.query.get(id)
            if not user:
                app.logger.error(f'User with id {id} not found...')
                return redirect(url_for('users'))
            form_action = request.args.get('action')
            if form_action == 'edit':
                return render_template('user_form.html', user=user)
            elif form_action == 'delete':
                db.session.delete(user)
                db.session.commit()
                return redirect(url_for('users'))
            return render_template('user_detail.html', user=user)
        else:
            return render_template('user_form.html', user=None)
    elif request.method == 'POST':
        id = request.form['id']
        first_name = request.form['first_name']
        last_name = request.form['last_name']
        username = request.form['username']
        email = request.form['email']
        if id:
            user = User.query.get(id)
            user.first_name = first_name
            user.last_name = last_name
            user.username = username
            user.email = email
        else:
            user = User(
                first_name=first_name,
                last_name=last_name,
                username=username,
                email=email
            )
            db.session.add(user)
        db.session.commit()
        return redirect(url_for('users'))
```

3. Run the application locally and validate by using http://localhost:5001/user?id=10. Check the logs generated in debug mode to see additional log messages.
4. Merge the changes into the remote main branch to deploy on to the VM.
5. Validate by using GET call with id that does not exist. Review the logs under **/var/log/sales-app** to see additional log messages.

* Exercise - Log Message for Course Not Found

1. Make sure to add log message for GET (select and delete) in case the course id does not exists.
2. Validate locally in debug mode by using course id that does not exists
3. Deploy on to the VM using CI/CD
4. Validate on Remote VM by using course id that does not exists
5. Review logs to see if the message is logged or not.